In [1]:
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from gurobipy import *
m1 = Model('vrptw')

# 20 customer -> 1.1 minute
# 30 customers -> 5 minute    1.69 minutes
# 40                          2.45 min
# 50 customer -> 160 minute


def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw

customers = 10

# timeWindows = [[20, 30], [30, 40], [30, 40]]

timeWindows = []
for i in range(0,customers):
    st = random.randint(1,2)*10
    et = random.randint(1,3)*10 + st
    tw = [st,et]
    timeWindows.append(tw)

print("time Windows ", timeWindows)
print()

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i, 10))

print("discreteTimeWindows", discreteTimeWindows)
print()
# starting_time = '6:00am'

# timeMatrix = [[0, 30, 10], [30, 0, 10], [10, 10, 0]]

timeMatrix = []
for i in range(0,customers):
    timeMatrix.append([0]*customers)
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
            timeMatrix[i][j] = random.randint(1,10)*10
            timeMatrix[j][i] = timeMatrix[i][j]

print("timeMatrix", timeMatrix)
print()


count = 0
nodeCount = []  # assigns a number to node using count and stores its value

for i in range(0, len(discreteTimeWindows)):
    c = []
    for j in range(0, len(discreteTimeWindows[i])):
        c.append(count+1)
        count += 1
    nodeCount.append(c)

print("Node Count ", nodeCount)
print()


edgeFlows = {}
edgeNumber = 0
x = [] 
edges = {}
edgesList = []  # stores edge number going out of ith node
edgesList.append([])  # for s
# solver = pywraplp.Solver.CreateSolver('GLOP')

# GRAPH CONSTRUCTION

# out of s to starting of each customer
e = []
for i in range(0, len(nodeCount)):
#     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
    x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
#     m1.update()
    edges['x[%i]' % edgeNumber] = (0, nodeCount[i][0])
    edgeFlows[edgeNumber] = (0, nodeCount[i][0])
    e.append(edgeNumber)
    edgeNumber += 1
edgesList[0] = e

for i in range(0, len(nodeCount)):
    for j in range(0, len(nodeCount[i])-1):
        e = []
#         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
#         m1.update()
        edges['x[%i]' % edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        edgeFlows[edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        # add edge number in edge list
        e.append(edgeNumber)
        edgesList.append(e)
        edgeNumber += 1
    edgesList.append([])
print("Edges List:", edgesList)

    
for i in range(len(discreteTimeWindows)):
    tu = nodeCount[i][-1]
    start_utime = discreteTimeWindows[i][0]
    end_utime = discreteTimeWindows[i][-1]

    # stores edges reachable from u
    for k in range(0, len(discreteTimeWindows)):
        sv = nodeCount[k][0]

        start_vtime = discreteTimeWindows[k][0]
        end_vtime = discreteTimeWindows[k][-1]


        if(i != k and start_utime + timeMatrix[i][k] > end_vtime):
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)

              # not reachable

        elif(i != k and start_vtime > end_utime + timeMatrix[i][k]):
            # edge bw (u,tu) -> (v,sv)
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
            
            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
            edgeFlows[edgeNumber] = (tu, sv)
            print(tu, "->", sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif(i != k and start_vtime == end_utime + timeMatrix[i][k]):
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
            

            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
            print(tu, "->", sv)
            edgeFlows[edgeNumber] = (tu, sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif (i != k and start_vtime < end_utime + timeMatrix[i][k]):
           
            firstReachableNode_index = 0  # first such that node >= sv
            for j in range(0, len(discreteTimeWindows[i])):
                if(timeMatrix[i][k] + discreteTimeWindows[i][j] >= start_vtime):
                    firstReachableNode_index = j
                    break
            # now starting from firstReachableNode make edges until feasible
            for l in range(firstReachableNode_index, len(discreteTimeWindows[i])):
                e = []
                reachingTime = discreteTimeWindows[i][l]+timeMatrix[i][k]
                for r in range(0, len(discreteTimeWindows[k])):
                    if(discreteTimeWindows[k][r] >= reachingTime):
                        print(edgesList[nodeCount[i][l]])
                        print(nodeCount[i][l], "->", nodeCount[k][r])
                        # edge bw these two
#                         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                        
                        edges['x[%i]' % edgeNumber] =(nodeCount[i][l], nodeCount[k][r])  # u -> v reachable
                        edgeFlows[edgeNumber] = (nodeCount[i][l], nodeCount[k][r])

                        e.append(edgeNumber)
                        edgesList[nodeCount[i][l]].extend(e)
                        print("hdhdh", edgesList[nodeCount[i][l]])
                        edgeNumber += 1
                        break




print('edge list b4 s t', edgesList)
print()

# into t (except s)
e = []
for i in range(0, len(nodeCount)):
#     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
    x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
    

    edges['x[%i]' % edgeNumber] = (nodeCount[i][-1], -1)
    edgeFlows[edgeNumber] = (nodeCount[i][-1], -1)
    # add edge number in edge list
    edgesList[nodeCount[i][-1]].append(edgeNumber)
    edgeNumber += 1
edgesList.append([])

m1.update()

print("Node Count after s and t:", nodeCount)
print()

# GRAPH Construction OVER
# if I add it to  0 edge i'll get flow for 1 at the same edge  , add to 1 customer 2 customer


offset = edgeNumber
print("edge flows:", edgeFlows)

flows = [0]*customers
for i in range(customers):
    flows[i] = edgeFlows.copy()
    keys = edgeFlows.keys()
    for j in keys:  # stores key values of edgeflows i.e 0 1 2
        # all x[] > offset are for flow variables
#         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
       
        # remove key and replace it by edge
        t = flows[i].pop(j)
        flows[i][edgeNumber] = t
        edgeNumber += 1

# print("flows", flows)
m1.update()

print('edge list', edgesList)
print()
print("edges", edges)
print()
print('edge number', edgeNumber)
print()

Restricted license - for non-production use only - expires 2023-10-25
time Windows  [[10, 30], [10, 40], [10, 40], [20, 30], [20, 40], [20, 30], [10, 20], [20, 40], [20, 40], [20, 40]]

discreteTimeWindows [[10, 20, 30], [10, 20, 30, 40], [10, 20, 30, 40], [20, 30], [20, 30, 40], [20, 30], [10, 20], [20, 30, 40], [20, 30, 40], [20, 30, 40]]

timeMatrix [[0, 60, 40, 40, 50, 70, 20, 70, 60, 80], [60, 0, 70, 80, 70, 100, 60, 30, 30, 80], [40, 70, 0, 100, 60, 50, 90, 50, 80, 40], [40, 80, 100, 0, 90, 60, 10, 70, 100, 60], [50, 70, 60, 90, 0, 30, 70, 60, 40, 30], [70, 100, 50, 60, 30, 0, 50, 60, 50, 100], [20, 60, 90, 10, 70, 50, 0, 50, 40, 30], [70, 30, 50, 70, 60, 60, 50, 0, 50, 60], [60, 30, 80, 100, 40, 50, 40, 50, 0, 70], [80, 80, 40, 60, 30, 100, 30, 60, 70, 0]]

Node Count  [[1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13], [14, 15, 16], [17, 18], [19, 20], [21, 22, 23], [24, 25, 26], [27, 28, 29]]

Edges List: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10], [11], [], [12], [13], [14], [], [

In [2]:
# # contraint for outgoing flow for a customer
for i in range(0, len(nodeCount)):  # choose one list from nodeCount
#     contraint = [0] * (edgeNumber)
    coeff = [] # store where coeff = 1
    for j in range(0, len(nodeCount[i])):  # se
        node = nodeCount[i][j]
        for k in range(0, len(edgesList[node])):
            e = edgesList[node][k]
            u = edges['x[%i]' % e][0]
            v = edges['x[%i]' % e][1]
                # print("u", u, "v", v)
            if(v not in nodeCount[i]):
                    # print('hello')
                    # print("u", u, "v", v)
                    coeff.append(x[e])
    m1.addConstr(( sum(coeff) == 1  ))



incomingEdgeList = []
for i in range(0, len(edgesList)):
    incomingEdgeList.append([])  # adding lists in a list
for i in edges:
    v = edges[i][1]  # u -> v

    edge_no = int(i[2:-1])
    incomingEdgeList[v].append(edge_no)

print("incoming edge list: ", incomingEdgeList)
print()

#  constraint for incoming for a customer
for i in range(0, len(nodeCount)):
    coeff = []
    contraint = [0] * (edgeNumber)  # indicates number of edges
    for j in range(0, len(nodeCount[i])):
        node = nodeCount[i][j]
        for k in range(0, len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            u = edges['x[%i]' % e][0]
            v = edges['x[%i]' % e][1]
                # print("u", u, "v", v)
            if(u not in nodeCount[i]):
                    # print('hello')
                    # print("u", u, "v", v)
                    coeff.append(x[e])
    m1.addConstr(( sum(coeff) == 1  ))
    
# constraint for a node incoming = outgoing
for i in range(0, len(nodeCount)):
    for j in range(0, len(nodeCount[i])):
        coeff = []
        contraint = [0] * (edgeNumber)
        node = nodeCount[i][j]
        for k in range(0, len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            contraint[e] = 1
            coeff.append(x[e])
        for k in range(0, len(edgesList[node])):
            e = edgesList[node][k]
            contraint[e] = -1
            coeff.append(-x[e])
        m1.addConstr(( sum(coeff) == 0 ))

        
        
    
# #  0<= xij - fij    f<x => 0<=x-f
for j in range(len(flows)):
    diff = offset*j+offset
    for k in flows[j].keys():
        contraint = [0]*edgeNumber
        contraint[k] = -1
        contraint[k-diff] = 1
        
        coeff = [] 
        coeff.append(-x[k])
        coeff.append(x[k-diff])
        m1.addConstr(( sum(coeff) >= 0 ))
        

#         # print(contraint," >= 0")

# # constraint for incoming for a particular customer = 1
# print("CONSTRAINT for incoming = 1 ")
for c in range(customers):
    addn = offset*(c+1)
    
    for i in range(0, len(nodeCount)):
        # indicates number of edges
        contraint = [0] * (edgeNumber)
        coeff = []
        for j in range(0, len(nodeCount[i])):
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                u = edges['x[%i]' % e][0]
                v = edges['x[%i]' % e][1]
                # print("u", u, "v", v)
                if(u not in nodeCount[i]):
                    # print('hello')
                    # print("u", u, "v", v)
                    contraint[e+addn] = 1
                    coeff.append(x[e+addn])
        m1.addConstr(( sum(coeff) == 1 ))
                    
                    


# # flow conservation for node
for c in range(customers):
    addn = offset*c+offset
    for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])):
            contraint = [0] * (edgeNumber)
            coeff = []
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                contraint[e+addn] = 1
                coeff.append(x[e+addn])
            for k in range(0, len(edgesList[node])):
                e = edgesList[node][k]
                contraint[e+addn] = -1
                coeff.append(-x[e+addn])
            m1.addConstr(( sum(coeff) == 0 ))

incoming edge list:  [[], [0], [10], [11, 31], [1], [12], [13], [14], [2], [15], [16], [17], [3, 32], [18, 33], [4], [19], [20], [5], [21], [6], [22], [7], [23], [24, 29], [8], [25], [26, 30], [9], [27], [28, 34], [35, 36, 37, 38, 39, 40, 41, 42, 43, 44]]



In [3]:
objectivefun = []
for i in range(0, len(edgesList[0])):
    objectivefun.append(x[edgesList[0][i]])
obj = sum(objectivefun)
m1.setObjective(obj,GRB.MINIMIZE)
                
    
m1.optimize()

print("objective value : ",m1.objVal)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads
Optimize a model with 889 rows, 495 columns and 1862 nonzeros
Model fingerprint: 0x1d84863f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 767 rows and 417 columns
Presolve time: 0.00s
Presolved: 122 rows, 78 columns, 337 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   6.200000e+01   0.000000e+00      0s
      33    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 33 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.000000000e+00
objective value :  8.0


In [4]:
offset

45

In [9]:
ans = []
for v in m1.getVars():
    a = []
    a.append(v.varName)
    a.append(v.x)
    b = []
    for j in a:
        b.append(j)
    ans.append(b)
ans = ans[0:offset]    
#     print('%s: %g' % (v.varName,v.x))
#     a.append

In [10]:
ans

[['x[0]', 1.0],
 ['x[1]', 1.0],
 ['x[2]', 1.0],
 ['x[3]', 0.0],
 ['x[4]', 1.0],
 ['x[5]', 1.0],
 ['x[6]', 1.0],
 ['x[7]', 0.0],
 ['x[8]', 1.0],
 ['x[9]', 1.0],
 ['x[10]', 1.0],
 ['x[11]', 1.0],
 ['x[12]', 0.0],
 ['x[13]', 0.0],
 ['x[14]', 0.0],
 ['x[15]', 1.0],
 ['x[16]', 1.0],
 ['x[17]', 1.0],
 ['x[18]', 0.0],
 ['x[19]', 1.0],
 ['x[20]', 1.0],
 ['x[21]', 1.0],
 ['x[22]', 1.0],
 ['x[23]', 0.0],
 ['x[24]', 0.0],
 ['x[25]', 1.0],
 ['x[26]', 1.0],
 ['x[27]', 1.0],
 ['x[28]', 1.0],
 ['x[29]', 1.0],
 ['x[30]', 0.0],
 ['x[31]', 0.0],
 ['x[32]', 0.0],
 ['x[33]', 1.0],
 ['x[34]', 0.0],
 ['x[35]', 1.0],
 ['x[36]', 0.0],
 ['x[37]', 1.0],
 ['x[38]', 1.0],
 ['x[39]', 1.0],
 ['x[40]', 1.0],
 ['x[41]', 0.0],
 ['x[42]', 1.0],
 ['x[43]', 1.0],
 ['x[44]', 1.0]]

In [11]:
possible_edges = list(edges.keys())
pair = list(edges.values())

In [13]:
flow_amt = []
for i in range(len(ans)):
    flow_amt.append(ans[i][1])

In [14]:
start_node = []
end_node = []
for i in range(len(pair)):
    start_node.append(pair[i][0])
    end_node.append(pair[i][1])

In [15]:
import numpy as np
s = np.unique(start_node)
e = np.unique(end_node)

In [18]:
outgoing_flow_amt = {}
for i in range(len(s)):

    c = 0.0
    for j in range(len(start_node)):
        if i == start_node[j]:
            c = c + flow_amt[j]
    
    outgoing_flow_amt[i] = c

In [19]:
outgoing_flow_amt

{0: 8.0,
 1: 1.0,
 2: 1.0,
 3: 1.0,
 4: 1.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 1.0,
 9: 1.0,
 10: 1.0,
 11: 1.0,
 12: 0.0,
 13: 1.0,
 14: 1.0,
 15: 1.0,
 16: 1.0,
 17: 1.0,
 18: 1.0,
 19: 1.0,
 20: 1.0,
 21: 0.0,
 22: 0.0,
 23: 1.0,
 24: 1.0,
 25: 1.0,
 26: 1.0,
 27: 1.0,
 28: 1.0,
 29: 1.0}

In [20]:
incoming_flow_amt = {}
for i in range(-1 ,len(e)):
    c = 0.0
    for j in range(len(end_node)):
        if i == end_node[j]:
            c = c + flow_amt[j]
    
    incoming_flow_amt[i] = c
del incoming_flow_amt[0]

In [21]:
incoming_flow_amt

{-1: 8.0,
 1: 1.0,
 2: 1.0,
 3: 1.0,
 4: 1.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 1.0,
 9: 1.0,
 10: 1.0,
 11: 1.0,
 12: 0.0,
 13: 1.0,
 14: 1.0,
 15: 1.0,
 16: 1.0,
 17: 1.0,
 18: 1.0,
 19: 1.0,
 20: 1.0,
 21: 0.0,
 22: 0.0,
 23: 1.0,
 24: 1.0,
 25: 1.0,
 26: 1.0,
 27: 1.0,
 28: 1.0,
 29: 1.0}

In [22]:
out = list(outgoing_flow_amt.values())
income = list(incoming_flow_amt.values())

In [23]:
type(out[0])

float

In [28]:
# the first element of income denotes the depot (source) 
# the first element of out denotes the sink
# so the outgoing out of source == incoming of sink
# and for each node, outgoing amount == incoming amout of flow
for i in range(len(out)):
    flag = 0
    if out[i] != income[i]:
        flag = 1
if flag == 1:
    print('The graph is not valid')
else:
    print('The graph is valid. The amount of flow going from source is %d and the amount of flow going into sink %d' %(out[0],income[0]))

The graph is valid. The amount of flow going from source is 8 and the amount of flow going into sink 8


In [29]:
edgesList

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10],
 [11],
 [35],
 [12, 29, 30],
 [13],
 [14],
 [36],
 [15],
 [16],
 [17],
 [37],
 [18],
 [38],
 [19],
 [20],
 [39],
 [21],
 [40],
 [22, 31, 32, 34],
 [33, 41],
 [23],
 [24],
 [42],
 [25],
 [26],
 [43],
 [27],
 [28],
 [44],
 []]

In [30]:
nodeCount

[[1, 2, 3],
 [4, 5, 6, 7],
 [8, 9, 10, 11],
 [12, 13],
 [14, 15, 16],
 [17, 18],
 [19, 20],
 [21, 22, 23],
 [24, 25, 26],
 [27, 28, 29]]